# Wedding FOM Example
T-Lind

Uses a configuration file found in the samples directory to compute the optimal policy for the wedding example.

In [1]:
import json
from timeit import default_timer as timer
import matplotlib.pyplot as plt

from ptcr2.fom import FOM

config_file = 'samples/wedding_fom.json'
# parse file name
file_name = config_file.split('/')[-1].split('.')[0]

with open(config_file) as f:
    spec = json.loads(f.read())


Computes the optimal policy. Prints out the time taken to compute the optimal policy as well as the configuration file used.

In [2]:
wedding_fom = FOM()
start = timer()
wedding_fom.compute_optimal_policy(spec)
end = timer()

print('Time elapsed to compute optimal policy: ', end - start)

Formatting checks succeeded.

            Transition Matrix            
┏━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃   ┃ I   ┃ E   ┃ B   ┃ C   ┃ D   ┃ S   ┃
┡━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│ I │ 0.0 │ 0.1 │ 0.3 │ 0.1 │ 0.2 │ 0.3 │
│ E │ 0.0 │ 0.1 │ 0.2 │ 0.1 │ 0.3 │ 0.3 │
│ B │ 0.0 │ 0.2 │ 0.1 │ 0.1 │ 0.3 │ 0.3 │
│ C │ 0.0 │ 0.1 │ 0.2 │ 0.2 │ 0.3 │ 0.2 │
│ D │ 0.0 │ 0.2 │ 0.3 │ 0.1 │ 0.1 │ 0.3 │
│ S │ 0.0 │ 0.4 │ 0.2 │ 0.2 │ 0.2 │ 0.0 │
└───┴─────┴─────┴─────┴─────┴─────┴─────┘

        Initial Distribution         
┏━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃ I   ┃ E   ┃ B   ┃ C   ┃ D   ┃ S   ┃
┡━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│ 0.0 │ 0.1 │ 0.3 │ 0.2 │ 0.2 │ 0.2 │
└─────┴─────┴─────┴─────┴─────┴─────┘

         Cost Matrix         
┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃ I ┃ E ┃ B ┃ C ┃ D ┃ S ┃
┡━━━╇━━━╇━━━╇━━━╇━━━╇━━━╇━━━┩
│ I │ 0 │ 1 │ 2 │ 3 │ 4 │ 5 │
│ E │ 1 │ 0 │ 1 │ 2 │ 3 │ 4 │
│ B │ 2 │ 1 │ 0 │ 1 │ 2 │ 3 │
│ C │ 3 │ 2 │ 1 │ 0 │ 1 │ 2 │
│ D │ 4 │ 3 │ 2 │ 1 │ 0 │ 1 │
│ S │ 5 │ 4 │ 3 │ 2 │ 1 │ 0 │
└───┴───┴───┴───┴───┴───┴───┘

Alphabet 
┏━━━━━━━┓
┃ Items ┃
┡━━━━━━━┩
│ s3    │
│ d2    │
│ b1    │
│ d12   │
│ d3    │
│ d23   │
│ b2    │
│ b3    │
│ c2    │
│ d1    │
│ e2    │
│ s2    │
│ s1    │
│ c1    │
│ c3    │
│ e3    │
│ e1    │
└───────┘

Single initial state 0:
[[('d1', 'd2'), 'd12']]

Single initial state 1:
[[('d2', 'd3'), 'd23']]

Adding v0 =  8_I_I_I
Unreachable state have been recognized
Start checking transitions to remove
Number of transitions: 2587213
End checking transitions to remove
Start removing unreachable transitions
End removing unreachable transitions
Start removing unreachable states
End removing unreachable states
Start reindexing states
End reindexing states
Number of unreachable states removed = 0
Number of unreachable transitions removed = 0
Time elapsed to compute optimal policy:  87.3408843


Saving the model that was generated to a file. This will allow for faster loading in the future.

In [3]:
fom_save_file_name = wedding_fom.save()
print('Model saved to: ', fom_save_file_name)

OSError: [Errno 22] Invalid argument: 'saves/C:\\Users\\tenant\\PycharmProjects\\cost_mechanisms_for_planning\\ptcr2\\fom.py_20240407-124847.pkl'

Loads the model from the file that was saved. This is useful for when you want to run simulations on the model without having to recompute the optimal policy.

In [ ]:
wedding_fom = FOM.load(fom_save_file_name)

Plots the training runs of the general algorithm. This can be helpful to determine how various factors might affect the training process.

In [ ]:
plt.title('Difference Measured While Computing Optimal Policy')
plt.xlabel('Iteration')
plt.ylabel('Difference')
plt.plot(wedding_fom.computed_policy['diff_tracker'], 'g', label='Run')
plt.axhline(wedding_fom.epsilon, linestyle='--', color='r', label='Epsilon of Convergence')
plt.legend()

Prints the result of one simulation run for both the general and greedy algorithm. This can be used to compare how much better our general algorithm is.

In [ ]:
results = wedding_fom.simulate_general_and_greedy_algorithms()  # Spec not needed since we're computing optimal policy separately

print(results)